# Streamfunction formulation

$$\textbf{u}=\nabla\times\boldsymbol{\psi} \iff \nabla\cdot\textbf{u}=0$$

$d=2$ Cartesian formulation

$$\textbf{u}=\nabla\times\psi\textbf{e}_z=-\frac{\partial\psi}{\partial y}\textbf{e}_x + \frac{\partial\psi}{\partial x}\textbf{e}_y$$

Stokes flow boundary value problem on $\textbf{x}\in\Omega$ 

$$\nabla\cdot\textbf{u} = 0$$
$$\textbf{0}=-\nabla p + \nabla^2\textbf{u} + \textbf{f}$$

Stokes flow streamfunction formulation

$$\textbf{f}=f_x\textbf{e}_x + f_y\textbf{e}_y$$

$$\nabla^2(\nabla^2\psi) = \frac{\partial f_y}{\partial x}- \frac{\partial f_x}{\partial y}$$

boundary conditions on $\textbf{x}\in\partial\Omega$ with unit normal $\textbf{n}\in\{\pm\textbf{e}_x, \pm\textbf{e}_y\}$ and unit tangent $\textbf{n}\cdot\textbf{t}=0$

$$\textbf{n}\cdot\textbf{u}=0 \iff \psi=0$$

$$\textbf{n}\cdot\nabla(\textbf{t}\cdot\textbf{u})=0 \iff \nabla^2\psi=0$$

variational formulation

$$F(v,\psi) = \dots$$

In [ ]:
import numpy as np
from ufl.core.expr import Expr
from ufl import (Form, FacetNormal, CellDiameter, dx, dS,
    TestFunction, TrialFunction, TestFunctions, TrialFunctions,
    inner, grad, div, avg, jump, Dx, as_matrix,
)

from lucifex.mesh import rectangle_mesh, mesh_boundary
from lucifex.fem import LUCiFExFunction as Function, LUCiFExConstant as Constant
from lucifex.solver import bvp_solver, interpolation_solver, BoundaryConditions, OptionsPETSc
from lucifex.utils import cross_section, fem_function_components
from lucifex.viz import plot_colormap, plot_line, plot_streamlines
from lucifex.io import write


def streamfunction_velocity(psi: Function) -> Expr:
    return as_matrix([[0, 1], [-1, 0]]) * grad(psi)


def stokes_streamfunction(
    psi: Function,
    f: Function | None = None,
    alpha: Constant | None = None,
) -> list[Form]:
    v = TestFunction(psi.function_space)
    psi_trial = TrialFunction(psi.function_space)
    n = FacetNormal(psi.function_space.mesh)
    h = CellDiameter(psi.function_space.mesh)

    F_dx = div(grad(v)) * div(grad(psi_trial)) * dx

    F_dS = (alpha / avg(h)) * inner(jump(grad(v), n), jump(grad(psi_trial), n)) * dS
    F_dS -= inner(jump(grad(v), n), avg(div(grad(psi_trial)))) * dS
    F_dS -= inner(avg(div(grad(v))), jump(grad(psi_trial), n)) * dS

    F_force = -v * Dx(f, 0) * dx

    return [F_dx, F_dS, F_force]


f = Function(
    (mesh, 'P', 1), 
    lambda x: 5 * x[1] * np.sin(6 * np.pi * x[0] / Lx),
    name='f',
)
alpha = Constant(mesh, 8.0, name='alpha')

bcs = BoundaryConditions(
    ('essential', boundary.union, 0.0),
)

psi = Function((mesh, 'P', 2), name="psi")
psi_solver = bvp_solver(stokes_streamfunction, bcs)(psi, f, alpha)
psi_solver.solve()

u = Function((mesh, 'P', 1, 2), name='u')
u_solver = interpolation_solver(u, streamfunction_velocity)(psi)
u_solver.solve()

In [ ]:
fig, ax = plot_colormap(psi, title='$\psi$', x_label='$x$', y_label='$y$')
write(fig, f'A07_stokes_flow_streamfunction', './figures', close=False, pickle=False)

ux, uy = fem_function_components(('P', 1), u, names=('ux', 'uy'))

x_axis, ux_x, y_value = cross_section(ux, 'y', 0.5)
plot_line((x_axis, ux_x), x_label='$y$', y_label=f'${u.name}_x(y={y_value:.2f})$')

x_axis, uy_x, y_value = cross_section(uy, 'y', 0.5)
plot_line((x_axis, uy_x), x_label='$y$', y_label=f'${u.name}_y(y={y_value:.2f})$')
